# MHKiT WEC-Sim Example

This example loads simulated data from a [WEC-Sim](http://wec-sim.github.io/WEC-Sim/index.html) run for a two-body point absorber [(Reference Model 3)](http://wec-sim.github.io/WEC-Sim/tutorials.html#two-body-point-absorber-rm3) and demonstrates the application of the [MHKiT wave module](https://mhkit-software.github.io/MHKiT/mhkit-python/api.wave.html) to interact with the simulated data. The analysis is broken down into three parts: load WEC-Sim data, view and interact with the WEC-Sim data , and apply the MHKiT wave module. 

  1. Load WEC-Sim Simulated Data
  2. WEC-Sim Simulated Data 
      - Wave Class Data
      - Body Class Data
      - PTO Class Data
      - Constraint Class Data
      - Mooring Class Data
  3. Apply MHKiT Wave Module
 
Start by importing MHKiT and the necessary python packages (e.g.`scipy.io` and `matplotlib.pyplot`).

In [ ]:
from  mhkit import wave
import scipy.io as sio
import matplotlib.pyplot as plt

## 1. Load WEC-Sim Simulated Data

WEC-Sim saves output data as a MATLAB output object, generated by WEC-Sim's [Response Class](http://wec-sim.github.io/WEC-Sim/api.html#response-class). The WEC-Sim output object must be converted to a structure for use in Python. 

Here we will load the WEC-Sim RM3 data run with a mooring matrix.

In [ ]:
# Relative location and filename of simulated WEC-Sim data (run with mooring)
filename = './data/wave/RM3MooringMatrix_matlabWorkspace_structure.mat' 

# Load data using the `wecsim.read_output` function which returns a dictionary of dataFrames
wecsim_data = wave.io.wecsim.read_output(filename)

**NOTE:** The `mhkit.wave.io.wecsim.read_output` function prints a message letting the user know which WEC-Sim classes were not used. This WEC-Sim example data for the RM3 was not run with MoorDyn, PTO-Sim or a cable. 

**NOTE**: Conversion of the WEC-Sim object  to a struct must be done *in MATLAB* and can be achieved using the command: `output = struct(output);`.


## 2. WEC-Sim Simulated Data

This section will investigate the WEC-Sim RM3 data loaded using MHKiT. In the previous section `mhkit.wave.io.wecsim.read_output` returned a [dictonary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) of [DataFrames](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe) in a  format similar to the WEC-Sim output object. To see what data is available we can call the `keys` method on the `wecsim_data` dictionary:

In [ ]:
# View the available WEC-Sim data, a dataFrame for each WEC-Sim Class
wecsim_data.keys()

We can see above that there are eight returned keys. As noted from the `mhkit.wave.io.wecsim.read_output` function the `moorDyn`, `ptosim` and `cables` keys will be empty as the function found the data missing from the output. The following sections will work through each of the 5 other WEC-Sim classes.

### Wave Class Data

Data from WEC-Sim's [Wave Class](http://wec-sim.github.io/WEC-Sim/code_structure.html#wave-class) includes information about the wave input, including the the wave type, and wave elevation as a function of time. Below we will wave the wave DataFrame as wave and then look at the wave type and the top (head) of the DataFrame.

In [ ]:
# Store WEC-Sim output from the Wave Class to a new dataFrame, called `wave_data`
wave_data = wecsim_data['wave']

# Display the wave type from the WEC-Sim Wave Class
wave_type = wave_data.name
print("WEC-Sim wave type:", wave_type)

# View the WEC-Sim output dataFrame for the Wave Class 
wave_data

The WEC-Sim wave type refers to the type of wave used to run the WEC-Sim simulation. Data from this WEC-Sim run was generated used '[etaImport](http://wec-sim.github.io/WEC-Sim/code_structure.html#etaimport)', meaning a user-defined wave surface elevation was defined. For more information about the wave types used by WEC-Sim, refer to the [Wave Class](http://wec-sim.github.io/WEC-Sim/code_structure.html#wave-class) documentation. 

#### Plot the Wave Elevation Data

We can use the pandas DataFrame plot method to quickly view the Data. The plot method will set the index (time) on the x-axis and any columns (in this case only wave elevation) on the y-axis.

In [ ]:
# Plot WEC-Sim output from the Wave Class
wave_data.plot()
plt.xlabel("Time [s]")
plt.ylabel("Wave Surface Elevation [m]")

### Body Class Data

Data from WEC-Sim's [Body Class](http://wec-sim.github.io/WEC-Sim/code_structure.html#body-class) includes information about each body, including the body's position, velocity, acceleration, forces acting on the body, and the body's name. For the RM3 example there will be 2 boides a float and a spar. 

In [ ]:
# Store WEC-Sim output from the Body Class to a new dictionary of dataFrames, i.e. 'bodies'. 
bodies = wecsim_data['bodies']

# Data fron each body is stored as its own dataFrame, i.e. 'body1' and 'body2'.
bodies.keys()

#### Body Class Data for Body 1

Let us determine which body 'body1' is by requesting the body's name. In this this case the body name is 'float'.

In [ ]:
# Store Body Class dataFrame for Body 1 as `body1`. 
body1 = bodies['body1']

# Display the name of Body 1 from the WEC-Sim Body Class
print("Name of Body 1:", body1.name)

For a given body WEC-Sim returns simulated parameters (the number of simulated parameters varies by the options choosen) for each of the 6 degrees of freedom (DOF). We can view the unique simulated parameters by filtering the columns by a single DOF. For this example we look at surge data (DOF 1).

In [ ]:
# Print a list of Body 1 columns that end with 'dof1'
[col for col in body1 if col.endswith('dof1')]

##### Plot heave position data for Body 1

The RM3 device creates power in the heave direction (DOF 3). Let us consider the float's (body 1) position as a function of time by plotting `postion_dof3`.

In [ ]:
# Use Pandas to plot Body 1 position in heave (DOF 3)
body1.position_dof3.plot()
plt.xlabel("Time [s]")
plt.ylabel("Heave Position [m]")
plt.title('Body 1')

# Use Pandas to calculate the maximum and minimum heave position of Body 1 
print("Body 1 max heave position =", body1.position_dof3.max(),"[m]")
print("Body 1 min heave position =", body1.position_dof3.min(),"[m]")

##### Plot Body 1 position data for all DOFs
As an example we could plot multiple postion DOFs by calling a plot on only the position columns. To do that we would first create a list of stings (`filter_col`) which include the string `'position'`. Then slicing the DataFrame on that list of strings and calling plot will plot the position over time of all 6 degrees of freedom for body1.

In [ ]:
# Create a list of Body 1 data columns that start with 'position'
filter_col = [col for col in body1 if col.startswith('position')]

# Plot filtered 'position' data for Body 1
body1[filter_col].plot()
plt.xlabel('Time [s]')
plt.ylabel('Position [m or rad]')
plt.title('Body 1')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

#### Body Class Data for Body 2

We can look at the name of body 2 to see that is the 'sapr' of RM3. Looking at the top of the body2 DataFrame we see the same available data as body1. 

In [ ]:
# Store Body Class dataFrame for Body 2 as `body2` 
body2 = bodies['body2']

# Display the name of Body 2 from the WEC-Sim Body Class
print("Name of Body 2:", body2.name)

# View the Body Class dataFrame for Body 2
body2.head()

### PTO Class Data

Data from WEC-Sim's [PTO Class](http://wec-sim.github.io/WEC-Sim/code_structure.html#pto-class) includes information about each PTO, including the PTO's position, velocity, acceleration, forces acting on the PTO, and the PTO's name. In this case there is only 1 PTO.

In [ ]:
# Store WEC-Sim output from the PTO Class to a DataFrame, called `ptos`
ptos = wecsim_data['ptos']

# Display the name of the PTO from the WEC-Sim PTO Class
print("Name of PTO:", ptos.name)

# Print a list of available columns that end with 'dof1'
[col for col in ptos if col.endswith('dof1')]

In [ ]:
# Use Pandas to plot pto internal power in heave (DOF 3)
# NOTE: WEC-Sim requires a negative sign to convert internal power to generated power
(-1*ptos.powerInternalMechanics_dof3/1000).plot()
plt.xlabel("Time [s]")
plt.ylabel("Power Generated [kW]")
plt.title('PTO')

### Constraint Class Data
Data from WEC-Sim's [Constraint Class](http://wec-sim.github.io/WEC-Sim/code_structure.html#constraint-class) includes  information about each constraint, including the constraint's position, velocity, acceleration, forces acting on the constraint, and the constraint's name. 

In [ ]:
# Store WEC-Sim output from the Constraint Class to a new dataFrame, called `constraints`
constraints = wecsim_data['constraints']

# Display the name of the Constraint from the WEC-Sim Constraint Class
print("Name of Constraint:", constraints.name)

# View the Constraint Class dataFrame
constraints.head()

### Mooring Class Data
Data from WEC-Sim's [Mooring Class](http://wec-sim.github.io/WEC-Sim/code_structure.html#mooring-class) includes relevant information about the mooring, including the mooring's position, velocity, mooring force, and the mooring's name. 

In [ ]:
# Store WEC-Sim output from the Mooring Class to a new dataFrame, called `mooring`
mooring = wecsim_data['mooring']

# View the PTO Class dataFrame
mooring.head()

## 3. Apply MHKiT Wave Module

In the example below we will calculate a spectrum from the wecsim wave elevation timeseries data using the MHKiT `elevation_spectrum` function.

In [ ]:
# Use the MHKiT Wave Module to calculate the wave spectrum from the WEC-Sim Wave Class Data
sample_rate=60
nnft=1000        # Number of bins in the Fast Fourier Transform
ws_spectrum = wave.resource.elevation_spectrum(wave_data, sample_rate, nnft)

# Plot calculated wave spectrum
spect_plot = wave.graphics.plot_spectrum(ws_spectrum)
spect_plot = spect_plot.set_xlim([0, 4])

In [ ]:
# Calculate Peak Wave Period (Tp) and Significant Wave Height (Hm0)
Tp = wave.resource.peak_period(ws_spectrum)
Hm0 = wave.resource.significant_wave_height(ws_spectrum)

# Display calculated Peak Wave Period (Tp) and Significant Wave Height (Hm0)
display(Tp,Hm0)